In [1]:
import wandb
import numpy as np
import torch
from pathlib import Path
from model_fusion.config import BASE_DATA_DIR, CHECKPOINT_DIR
from model_fusion.datasets import DataModuleType
from model_fusion.models import ModelType
from model_fusion.models.lightning import BaseModel 
from Experiments import lmc_experiment
from Experiments import baselines_experiment
from Experiments import otfusion_experiment
from Experiments import pyhessian_experiment
from model_fusion.train import setup_training


# set seed for numpy based calculations
NUMPY_SEED = 100
np.random.seed(NUMPY_SEED)

/Users/alecabodi/ethz_model_fusion/.venv/lib/python3.11/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/alecabodi/ethz_model_fusion/.venv/lib/python3.11/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/alecabodi/ethz_model_fusion/.venv/lib/python3.11/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/Users/alecabodi/ethz_model_fusion/.v

In [ ]:
print("------- Loading models -------")

# select wandb run names
runA = 'bbecqkxs'
runB = 'k9q16yq1'

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runA}')

print(run.config)

batch_size = run.config['datamodule_hparams'].get('batch_size')

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = run.config['datamodule_hparams']

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

# model_hparams = list(filter(lambda x: 'model_hparams/' in x[0], run.config.items()))
# model_hparams = {k.split('/')[1]: v for k, v in model_hparams}

model_hparams = run.config['model_hparams']

print(datamodule_hparams)
print(model_hparams)

checkpointA = f'model-fusion/Model Fusion/model-{runA}:best_k'
checkpointB = f'model-fusion/Model Fusion/model-{runB}:best_k'

run = wandb.init()

artifact = run.use_artifact(checkpointA, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelA = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

torch.save(modelA.model.state_dict(), 'model0.ckpt')

artifact = run.use_artifact(checkpointB, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelB = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

torch.save(modelB.model.state_dict(), 'model1.ckpt')


In [ ]:
# LMC barrier
print("------- Computing LMC barrier -------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA,
    modelB=modelB,
)

In [ ]:
# Baselines (prediction ensembling, vanilla averaging)
print("------- Computing baselines -------")

wandb_tag = f'baselines-{runA}-{runB}'

vanilla_averaging_model = baselines_experiment.run_baselines(
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
)

In [ ]:
# OT model fusion + eval aligned model 
print("------- Computing model fusion -------")

wandb_tag = f"ot_model_fusion-{runA}-{runB}"

ot_fused_model, modelA_aligned = otfusion_experiment.run_otfusion(
    batch_size=batch_size,
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag
)

In [ ]:
# Pyhessian I (compute sharpness and eigenspectrum of base models, vanilla avg and ot fusion solutions)
print("------- Computing sharpness -------")

print("------- Model A -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA)

print("------- Model B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelB)

print("------- Model A aligned to B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA_aligned)

print("------- OT fusion model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=ot_fused_model)

print("------- Vanilla avg model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=vanilla_averaging_model)


In [ ]:
# finetuning
# TODO
_, datamodule, trainer = None, None, None

datamodule.prepare_data()

datamodule.setup('fit')
trainer.fit(ot_fused_model, train_dataloaders=datamodule.train_dataloader(), val_dataloaders=datamodule.val_dataloader())
trainer.fit(vanilla_averaging_model, train_dataloaders=datamodule.train_dataloader(), val_dataloaders=datamodule.val_dataloader())

datamodule.setup('test')
trainer.test(ot_fused_model, dataloaders=datamodule.test_dataloader())
trainer.test(vanilla_averaging_model, dataloaders=datamodule.test_dataloader())

wandb.finish()

In [ ]:
# Comparison II (compute sharpness of finetuned solutions)
print("------- Computing sharpness of finetuned solutions -------")

print("------- OT fusion model (finetuned) -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=ot_fused_model)

print("------- Vanilla avg model (finetuned) -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=vanilla_averaging_model)
